In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, shutil, sqlite3
import pandas as pd
import numpy as np

RAW_DB = "/content/drive/MyDrive/is5126/data/reviews_sample.db"
CLEAN_DB = "/content/drive/MyDrive/is5126/data/reviews_clean.db"

# Create a fresh cleaned DB copy (safe: doesn't touch raw)
shutil.copy(RAW_DB, CLEAN_DB)

conn = sqlite3.connect(CLEAN_DB)
cursor = conn.cursor()


Baseline: size before profiling

In [3]:
baseline = pd.read_sql("""
SELECT
  COUNT(*) AS n_rows,
  COUNT(DISTINCT offering_id) AS n_hotels,
  COUNT(DISTINCT CASE WHEN author_id IS NOT NULL AND author_id<>'' THEN author_id END) AS n_authors
FROM reviews;
""", conn)

baseline


,n_rows,n_hotels,n_authors
0,754798,3888,522888


In [4]:
invalid_before = pd.read_sql("""
SELECT COUNT(*) AS invalid_overall
FROM reviews
WHERE overall < 1 OR overall > 5 OR overall IS NULL;
""", conn)
invalid_before

cursor.execute("""
DELETE FROM reviews
WHERE overall < 1 OR overall > 5 OR overall IS NULL;
""")
conn.commit()

invalid_after = pd.read_sql("""
SELECT COUNT(*) AS invalid_overall
FROM reviews
WHERE overall < 1 OR overall > 5 OR overall IS NULL;
""", conn)
invalid_after



,invalid_overall
0,0


Exact duplicate removal (hotel + author + text)

In [5]:
cursor.executescript("""
DROP TABLE IF EXISTS reviews_dedup_exact;

CREATE TABLE reviews_dedup_exact AS
SELECT r.*
FROM reviews r
JOIN (
  SELECT
    offering_id,
    author_id,
    text,
    MIN(id) AS keep_id
  FROM reviews
  WHERE author_id IS NOT NULL AND author_id <> ''
    AND text IS NOT NULL AND text <> ''
    AND review_date IS NOT NULL
  GROUP BY offering_id, author_id, text
) k
ON r.id = k.keep_id;
""")
conn.commit()


In [6]:
counts = pd.read_sql("""
SELECT
  (SELECT COUNT(*) FROM reviews) AS before_rows,
  (SELECT COUNT(*) FROM reviews_dedup_exact) AS after_rows;
""", conn)

counts


,before_rows,after_rows
0,754781,710122


**Keep one review per author-hotel**

We keep the longest text per (offering_id, author_id) to retain maximum signal.

In [7]:
cursor.executescript("""
DROP TABLE IF EXISTS reviews_dedup_author_hotel;

CREATE TABLE reviews_dedup_author_hotel AS
SELECT *
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY offering_id, author_id
      ORDER BY LENGTH(COALESCE(text,'')) DESC, review_date DESC
    ) AS rn
  FROM reviews_dedup_exact
  WHERE author_id IS NOT NULL AND author_id <> ''
    AND offering_id IS NOT NULL
    AND review_date IS NOT NULL
) t
WHERE rn = 1;
""")
conn.commit()


In [8]:
counts2 = pd.read_sql("""
SELECT
  (SELECT COUNT(*) FROM reviews_dedup_exact) AS before_rows,
  (SELECT COUNT(*) FROM reviews_dedup_author_hotel) AS after_rows;
""", conn)

counts2


,before_rows,after_rows
0,710122,703320


Remove low-information reviews (word count threshold)

In [9]:
MIN_WORDS = 20

cursor.executescript(f"""
DROP TABLE IF EXISTS reviews_final;

CREATE TABLE reviews_final AS
SELECT *
FROM reviews_dedup_author_hotel
WHERE
  -- basic text requirements
  text IS NOT NULL AND text <> ''
  -- word count approximation: count spaces + 1
  AND (LENGTH(TRIM(text)) - LENGTH(REPLACE(TRIM(text), ' ', '')) + 1) >= {MIN_WORDS};
""")
conn.commit()


Confirm final size (and ≥ 50k)

In [10]:
final_counts = pd.read_sql("""
SELECT
  COUNT(*) AS n_reviews_final,
  COUNT(DISTINCT offering_id) AS n_hotels_final,
  COUNT(DISTINCT author_id) AS n_authors_final
FROM reviews_final;
""", conn)

final_counts


,n_reviews_final,n_hotels_final,n_authors_final
0,685149,3863,510065


In [11]:
cursor.executescript("""
DROP TABLE IF EXISTS hotel_features;

CREATE TABLE hotel_features AS
SELECT
  offering_id,
  COUNT(*) AS n_reviews,
  AVG(overall) AS avg_overall,
  AVG(service) AS avg_service,
  AVG(cleanliness) AS avg_cleanliness,
  AVG(value) AS avg_value,
  AVG(location_rating) AS avg_location,
  -- variance: E[x^2] - (E[x])^2
  (AVG(overall*overall) - AVG(overall)*AVG(overall)) AS var_overall,

  -- aspect counts (important for stability)
  SUM(service IS NOT NULL) AS n_service,
  SUM(cleanliness IS NOT NULL) AS n_cleanliness,
  SUM(value IS NOT NULL) AS n_value,
  SUM(location_rating IS NOT NULL) AS n_location

FROM reviews_final
GROUP BY offering_id;
""")
conn.commit()


In [12]:
hotel_features = pd.read_sql("SELECT * FROM hotel_features;", conn)
hotel_features.describe()


,offering_id,n_reviews,avg_overall,avg_service,avg_cleanliness,avg_value,avg_location,var_overall,n_service,n_cleanliness,n_value,n_location
count,3.863000e+03,3863.000000,3863.000000,3850.000000,3849.000000,3851.000000,3839.000000,3863.000000,3863.000000,3863.000000,3863.000000,3863.000000
mean,4.140638e+05,177.361895,3.509280,3.723402,3.781135,3.651868,3.961136,1.210446,167.253689,166.767797,166.955734,153.575977
std,5.754102e+05,293.200841,0.887530,0.845654,0.916715,0.780539,0.736403,0.629597,276.302448,275.516118,275.537032,253.725207
min,7.257200e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.530100e+04,18.000000,3.056904,3.384816,3.339286,3.384615,3.628154,0.806818,17.000000,17.000000,17.000000,16.000000
50%,1.219870e+05,59.000000,3.743802,3.970369,4.066667,3.848039,4.118644,1.160998,56.000000,56.000000,56.000000,52.000000
75%,4.983255e+05,213.000000,4.142857,4.302628,4.445312,4.164042,4.500000,1.555556,200.000000,201.000000,200.500000,184.000000
max,3.574675e+06,2958.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,2724.000000,2743.000000,2734.000000,2592.000000


In [13]:
hotel_features_stable = hotel_features[
    (hotel_features["n_reviews"] >= 50) &
    (hotel_features["n_service"] >= 20) &
    (hotel_features["n_cleanliness"] >= 20) &
    (hotel_features["n_value"] >= 20) &
    (hotel_features["n_location"] >= 20)
].copy()

hotel_features_stable.shape


(2089, 12)

In [14]:
conn.commit()   # safe even if no pending writes
conn.close()
